In [51]:
from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown
from dotenv import load_dotenv
from datetime import datetime
from pydantic import BaseModel, Field
from typing import Dict, Any
import requests
import asyncio
import os

In [ ]:
load_dotenv(override=True)
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")
print(PERPLEXITY_API_KEY[:5] + "...")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:8] + "...")

In [ ]:
# in format "June 2025"
current_date = datetime.now().strftime("%B %Y")
print(current_date)


### Initial Researcher

In [56]:
INITIAL_RESEARCH_INSTRUCTIONS = """
 You are a research assistant. 
 Given a search term, you search the web for that term and produce a concise report of the results at the current date - {current_date}. 
 The summary must 2-3 paragraphs and less than 500 words. 
 Write succintly, no need to have complete sentences or good grammar. 
 This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. 
 Do not include any additional commentary other than the summary itself.

 Report should capture the main points: 
 ## Company Snapshot (1-2 paragraphs)
 ## Development Timeline (major milestones)
 ## Key People (founders, team, advisors, etc.)
 ## Products & Technology (key productes, servises, offers and technologies with description and main use cases)
 ## Business Model & Monetization (how they make money: payment plans, B2B/B2C channels, unit-economics)
 ## Tokenomics (if applicable)
 ## Market Position & Competition (Target Segments, Benchmarked competitors, Competitive Moat & KPIs)
 ## Partnerships & Clients (list of Tier-1 partners, integrations, cast-offs)
 ## Roadmap & Strategic Orientation (if available)
"""

initial_research_agent = Agent(
    name="Initial research agent",
    instructions=INITIAL_RESEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
message = "Broxus (https://www.broxus.com/)"

with trace("DeepResearch"):
    init_research_result = await Runner.run(initial_research_agent, message)

display(Markdown(init_research_result.final_output))

### Planner

In [40]:
PLANNER_INSTRUCTIONS = f"""
You are Principal Research Lead.
Your goal is to create a detailed plan for subagents based on the initial research report (<initial_research_report>) from Search agent (date of analysis - {current_date}). 
Your task is to analyze the report, identify the key topics and create a detailed plan for subagents, which will cover all the critical aspects.

**Контекст:**  
Research Leader provided a brief report (<initial_research_report>) with basic information about the project or person, including description, positioning, key people, metrics, and partnerships. 
Your task is to turn this overview into a detailed research plan, covering all critical aspects and eliminating the identified gaps.

**Instructions (Chain of Thoughts):**

**Step 1. Analysis of the provided report**
Extract the key blocks from the report:
 ## Company Snapshot
 ## Development Timeline
 ## Key People 
 ## Products & Technology
 ## Business Model & Monetization
 ## Tokenomics
 ## Market Position & Competition
 ## Partnerships & Clients
 ## Roadmap & Strategic Orientation

**Step 2. Formulating sections (multi-agent approach):**  
For each section:
- Write a detailed `research_prompt` queries that will help fill the gaps in the <initial_research_report> (in the style of a manual), including:  
  • specific tasks and questions,  
  • connection with information from the initial report,  
  • types of data to find (with examples),  
  • requirement to update data on - {current_date},  
  • instructions for formatting sources,  
  • explanation of how this block is related to the overall research  

  For each deficiency:
  - Break down the missing elements into specific searchable items
  - Create targeted search queries that will yield relevant results
  - Include clear reasoning for each search query
  - Focus on authoritative sources and recent information


**Step 3. Verification and structuring (Chain of Verification):**  
- Make sure each section contains a complete and meaningful `prompt`  
- The title and questions correspond to the content  
- There are no less than 3 and no more than 10 questions
- Specify the requirements for sources  

**Restrictions:**  
- Do not use generalized/vague formulations   
- Each block must be self-sufficient, complete, and ready for the assistant to work  

**Important:**  
For search, use the most up-to-date information on the current date - {current_date}. 
At the beginning and end of the work, remind yourself of the task goal — to create a **structured, reproducible, and scalable plan**, 
sufficient for launching research work within the team.
"""

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:
with trace("DeepResearch"):
    planner_result = await Runner.run(planner_agent, init_research_result.final_output)
    print(planner_result.final_output)

In [ ]:
for i in planner_result.final_output.searches:
    print(i)

### Deep Researcher

In [43]:
@function_tool
def get_perplexity_response(query: str) -> str:
    """Perform a search on the web using Perplexity API and return the response content.
    Args:
        query: The query to search for.
    Returns:
        The content of the search result as a string.
    """

    perplexity_system_prompt = f"""
    You are a helpful AI assistant
    Rules:
    1. Provide only the final answer. It is important that you do not include any explanation on the steps below.
    2. Do not show the intermediate steps information.
    3. For search, use the most up-to-date information on the current date - {current_date}.
    Steps:
    1. Decide if the answer should be a brief sentence or a list of suggestions.
    2. If it is a list of suggestions, first, write a brief and natural introduction based on the original query.
    3. Followed by a list of suggestions, each suggestion should be split by two newlines.
    """
    
    url = "https://api.perplexity.ai/chat/completions"

    payload = {
        "model": "sonar",
        "messages": [
            {
                "role": "system",
                "content": perplexity_system_prompt
            },
            {
                "role": "user",
                "content": query
            }
        ]
    }
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=payload).json()

    # Добавим проверку на ошибки от API
    if 'error' in response:
        return f"API Error: {response['error'].get('message', 'Unknown error')}"

    # Безопасно извлекаем контент
    try:
        content = response['choices'][0]['message']['content']
        citation = response['search_results']
        output = {
            "content": content,
            "citation": citation
        }
    except (KeyError, IndexError):
        return "Error: Could not parse the response from the API."

    return output

In [ ]:
get_perplexity_response

In [64]:
INSTRUCTIONS = """
 You are a research assistant. 
 Given a search term, by using get_perplexity_response, you search the web for that term and produce a concise report of the results. 
 Use the get_perplexity_response tool at least 2 times.
 Write succintly, no need to have complete sentences or good grammar. 
 This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. 
 Do not include any additional commentary other than the summary itself.
 
 Citation:
 Ensure that each fact in the article is supported by an embedded citation, all links are accurate and active, and the article ends with a single, correct, and sorted "Sources" section.
- Ensure that **each factual statement** is supported by an embedded citation (e.g., `[Source](URL)`), confirming exactly that fact.  
- It is allowed to format the link in the text through formulations: "according to [source](URL)", "in accordance with [source](URL)" and others.  

 Best Practices for Prompting Web Search Models:
 - Be Specific and Contextual: Unlike traditional LLMs, our web search models require specificity to retrieve relevant search results. Adding just 2-3 extra words of context can dramatically improve performance.
    - Good Example: “Explain recent advances in climate prediction models for urban planning”
    - Poor Example: “Tell me about climate models”
 - Avoid Few-Shot Prompting: While few-shot prompting works well for traditional LLMs, it confuses web search models by triggering searches for your examples rather than your actual query.
    - Good Example: “Summarize the current research on mRNA vaccine technology”
    - Poor Example: “Here’s an example of a good summary about vaccines: [example text]. Now summarize the current research on mRNA vaccines.”
 - Think Like a Web Search User: Craft prompts with search-friendly terms that would appear on relevant web pages. Consider how experts in the field would describe the topic online.
    - Good Example: “Compare the energy efficiency ratings of heat pumps vs. traditional HVAC systems for residential use”
    - Poor Example: “Tell me which home heating is better”
 - Provide Relevant Context: Include critical context to guide the web search toward the most relevant content, but keep prompts concise and focused.
    - Good Example: “Explain the impact of the 2023 EU digital markets regulations on app store competition for small developers”
    - Poor Example: “What are the rules for app stores?”
"""

tools=[get_perplexity_response]

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=tools,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
example = planner_result.final_output.searches[0]
query = "The query is: " + example.query + "\n\n" + "The reason of the query is: " + example.reason
query


In [ ]:
with trace("Search"):
    deep_search_result = await Runner.run(search_agent, query)

display(Markdown(deep_search_result.final_output))

### Writer

In [58]:
WRITER_INSTRUCTIONS = ("""
    You are a senior researcher tasked with writing a cohesive report for a research query. 
    You will be provided with the original query, and some initial research done by a research assistant.
    You should first come up with an outline for the report that describes the structure and 
    flow of the report. Then, generate the report and return that as your final output.
    The final output should be in markdown format, and it should be lengthy and detailed. 
    Aim for 3-4 pages of content, at least 1000 words.
                
    For the report, follow the structure:
      ## Steps of execution (managed chain of verification):

      **Step 1. Language correction**  
      - Correct grammatical, punctuation, spelling, and syntactic errors.  
      - Ensure consistency in case, tense, subject-verb agreement, and correct endings.

      **Step 2. Improvement of style and readability**  
      - Rewrite complex or heavyweight phrases for better clarity.  
      - Improve the coherence between sentences and paragraphs.  
      - Remove repetitions, redundant introductory constructions, and bureaucratic language.
      - Ensure that the description of the topic is fully developed and there are no gaps in understanding.

      **Step 3. Maintaining a professional tone**  
      - Maintain a consistent, professional, and analytical-neutral style.  
      - Avoid conversational or inappropriate emotional expressions.

      **Step 4. Verification (Chain of Verification)**  
      After making changes, ensure:  
      - Citations (in the format `[Text](URL)`) remain **unchanged**  
      - No numerical values, facts, names, dates, projects, organizations were distorted  
      - The logic of citation and the fact-to-source link is preserved  
      - Any formulations explicitly taken from the source remain unchanged  
      ❗ If you accidentally made changes to the citation or fact, return the original formulation.

      **Reminder:**  
      Your work is **exclusively** about improving the language and style.  
      **No facts, numbers, citations, sources, hyperlinks should be changed.**

    Mandatory formatting:
      **1. Semantic structure of headings**  
      - Check the hierarchy of headings:  
        `#` — document title, `##` — main sections, `###` — subsections.  
      - Ensure the logical structure of the document: no missing levels, nesting is observed.

      **2. Content formatting**  
      - Check that all lists are formatted correctly (`-` or `1.`), nested lists are readable.  
      - Quotes are formatted with `>` or embedded links `[text](URL)`.  
      - Code blocks (if any) are wrapped in triple backticks `` ``` `` with the language (if applicable).  
      - Subheadings, paragraphs, intervals — formatted uniformly.

      **3. Natural density of key phrases**  
      - Ensure that key phrases and terms are not violated by formatting or breaks in Markdown.  
      - Maintain readability — do not insert links inside key words if this distorts the meaning.

      **4. Sources section**  
      - Check that the `## Sources` section is at the very end.  
      - Each link in it is formatted as:  
        `[Author or publication – Article title](URL) (Date accessed: YYYY-MM-DD)`.  
      - Ensure that the list is sorted alphabetically and does not contain duplicates.
     
    Capture the main points. 
    # Company Snapshot (1-2 paragraphs)
    # Development Timeline (major milestones)
    # Products & Technology (key productes, servises, offers and technologies)
    # Business Model & Monetization (how they make money: payment plans, B2B/B2C channels, unit-economics)
    # Tokenomics (if applicable)
    # Market Position & Competition (Target Segments, Benchmarked competitors, Competitive Moat & KPIs)
    # Partnerships & Clients (list of Tier-1 partners, integrations, cast-offs)
    # Roadmap & Strategic Outlook (public plans + expert assessment of realism)"""
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")


writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
with trace("Writer"):
    result = await Runner.run(writer_agent, deep_search_result.final_output)

display(Markdown(result.final_output.short_summary))

In [ ]:
display(Markdown(result.final_output.short_summary))

## Putting all together

#### Researching

In [61]:
async def initial_research(query: str):
    """ Use the initial_research_agent to perform the initial research to get the draft of the report """
    print("Initial researches...")
    result = await Runner.run(initial_research_agent, f"Query: {query}")
    return result.final_output

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

#### Report Creation

In [62]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output


In [ ]:
query = "Perform a comprehensive analysis of the company Broxus (https://broxus.com/)"

with trace("Deep Research trace"):
    print("Starting research...")
    initial_research = await initial_research(query)
    search_plan = await plan_searches(initial_research)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    display(Markdown(report.markdown_report))
